In [153]:
import numpy as np
import re
from itertools import product, permutations, combinations, cycle, zip_longest
from collections import Counter, deque, defaultdict, OrderedDict
import networkx as nx
import string
import matplotlib.pyplot as plt
from functools import cmp_to_key

%matplotlib inline

In [243]:
# input
problemnumber = '16'
fn = f'../{problemnumber}/{problemnumber}.txt'
# fn = f'../{problemnumber}/{problemnumber}t.txt'

G = nx.Graph()
frs = {}
with open(fn) as fh:
    for l in fh.readlines():
        v = l[6:8]
        fr = re.findall(r'flow rate=(\d+)', l)[0]
        frs[v] = int(fr)
        adj = re.findall(r'valve[s]* ([A-Z\,?\s?]*)', l.rstrip())[0]
#         adj = re.findall(r'valves ([A-Z]*)', l.rstrip())
        adj = adj.replace(',', '').split()
#         print(adj)
        for a in adj:
            G.add_edge(v, a)

frs = {fr:frs[fr] for fr in sorted(frs, key=frs.get)[-1::-1]}

In [244]:
fr_nonz = {k:v for k, v in frs.items() if v>0}

{'PM': 25,
 'XO': 23,
 'VI': 22,
 'II': 20,
 'KQ': 18,
 'TV': 17,
 'AS': 16,
 'HR': 14,
 'FQ': 12,
 'RU': 11,
 'MA': 10,
 'ED': 9,
 'MW': 7,
 'LF': 5,
 'DW': 4}

In [247]:
# calc all distances
valve_dists = {}
for v1, v2 in combinations(fr_nonz.keys(), 2):
    dist = nx.shortest_path_length(G, v1, v2)
    valve_dists[(v1, v2)] = dist
    valve_dists[(v2, v1)] = dist
for v2 in fr_nonz.keys():
    v1 = 'AA'
    dist = nx.shortest_path_length(G, v1, v2)
    valve_dists[(v1, v2)] = dist
    valve_dists[(v2, v1)] = dist

In [250]:
fr_nonz
[print(l2, v) for (l1, l2), v in valve_dists.items() if l1=='AA']

PM 9
XO 5
VI 7
II 4
KQ 10
TV 5
AS 4
HR 2
FQ 5
RU 7
MA 2
ED 6
MW 3
LF 3
DW 2


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [239]:
# run all possible orders, memoize
memo = {} #order: (t, pressure)
memo[('AA', )] = (0, 0)

test_list = [('DD', 'BB', 'JJ', 'HH', 'EE', 'CC'),
             ('DD', 'BB', 'JJ', 'HH', 'CC', 'EE')]

maxp = 0

t, p = 0, 0
for order in permutations(fr_nonz.keys(), len(fr_nonz)):
# for order in test_list:
    order = ('AA',) + order
    found = False
    #check memo
    for o_ind in reversed(range(1, len(order) + 1)):
        sub_order = order[:o_ind]
        if sub_order in memo:
            t, p = memo[sub_order]
            current = sub_order
            remaining = deque(order[o_ind:])
            break
    
    l0 = current[-1]
    path = list(current)
#     print(current, remaining)
    while remaining:
        l1 = remaining.popleft()
        path.append(l1)
        t += valve_dists[(l0, l1)] + 1
        if t >= 30:
            break
        p += fr_nonz[l1] * (30 - t)
        memo[tuple(path)] = (t, p)
        l0 = l1
    
#     print(p)
    if p > maxp:
        maxp = p
        print(maxp)

1240
1240
1247
1247
1220
1213
1240
1184
1229
1226
1172
1114
1218
1161
1163
1156
1072
1058
1072
1051
1150
1126
1135
1078
1069
982
1395
1395
1402
1402
1339
1332
1359
1303
1397
1394
1303
1254
1345
1303
1327
1290
1325
1276
1241
1311
1310
1254
1386
1296
1238
1240
1231
1228
1174
1116
1170
1113
1258
1255
1216
1167
1206
1164
1040
982
1159
1110
933
1052
1121
1064
1240
1198
1022
1141
1239
1232
1218
1204
1148
1127
1388
1351
1386
1337
1302
1372
1220
1162
1253
1204
1113
1232
1099
1078
1244
1246
1188
1225
1101
1077
1086
1029
1020
933
1252
1196
1254
1164
1180
1182
1202
1145
1233
1191
1015
1134
978
957
1123
1125
959
996
1437
1437
1444
1444
1417
1410
1437
1381
1350
1347
1293
1235
1339
1282
1436
1429
1345
1331
1345
1324
1347
1323
1332
1275
1266
1179
1595
1595
1602
1602
1611
1611
1604
1595
1539
1612
1612
1599
1611
1515
1611
1610
1610
1603
1539
1539
1455
1532
1515
1523
1604
1603
1522
1459
1477
1474
1470
1412
1416
1359
1645
1645
1632
1644
1548
1590
1589
1469
1411
1539
1491
1275
1433
1462
1451
1626
1625
145

{'HH': 22, 'JJ': 21, 'DD': 20, 'BB': 13, 'EE': 3, 'CC': 2}

In [241]:
memo

{('AA',): (0, 0),
 ('AA', 'HH'): (6, 528),
 ('AA', 'HH', 'JJ'): (14, 864),
 ('AA', 'HH', 'JJ', 'DD'): (18, 1104),
 ('AA', 'HH', 'JJ', 'DD', 'BB'): (21, 1221),
 ('AA', 'HH', 'JJ', 'DD', 'BB', 'EE'): (25, 1236),
 ('AA', 'HH', 'JJ', 'DD', 'BB', 'EE', 'CC'): (28, 1240),
 ('AA', 'HH', 'JJ', 'DD', 'BB', 'CC'): (23, 1235),
 ('AA', 'HH', 'JJ', 'DD', 'BB', 'CC', 'EE'): (26, 1247),
 ('AA', 'HH', 'JJ', 'DD', 'EE'): (20, 1134),
 ('AA', 'HH', 'JJ', 'DD', 'EE', 'BB'): (24, 1212),
 ('AA', 'HH', 'JJ', 'DD', 'EE', 'BB', 'CC'): (26, 1220),
 ('AA', 'HH', 'JJ', 'DD', 'EE', 'CC'): (23, 1148),
 ('AA', 'HH', 'JJ', 'DD', 'EE', 'CC', 'BB'): (25, 1213),
 ('AA', 'HH', 'JJ', 'DD', 'CC'): (20, 1124),
 ('AA', 'HH', 'JJ', 'DD', 'CC', 'BB'): (22, 1228),
 ('AA', 'HH', 'JJ', 'DD', 'CC', 'BB', 'EE'): (26, 1240),
 ('AA', 'HH', 'JJ', 'DD', 'CC', 'EE'): (23, 1145),
 ('AA', 'HH', 'JJ', 'DD', 'CC', 'EE', 'BB'): (27, 1184),
 ('AA', 'HH', 'JJ', 'BB'): (18, 1020),
 ('AA', 'HH', 'JJ', 'BB', 'DD'): (21, 1200),
 ('AA', 'HH', 'JJ',

In [152]:
maxans = 0
memo = {} # path: (ans, t)

test_list = [('DD', 'BB', 'JJ', 'HH', 'EE', 'CC')]

def do_valve_order(order, G, fr_nonz):
    ans = 0
    t = 0
    switched = set()
    start = 'AA'
    for o in order:
        path = nx.shortest_path(G, start, o)
        for p in path[1:]:
#             print(p)
            t += 1
            if t == 30:
                return ans
#             if p in fr_nonz and p not in switched:
            if p==o:
                t += 1
                ans += fr_nonz[p] * (30 - t )
#                 print(p, t, fr_nonz[p])
                switched.add(p)
        start = o
    return ans

for order in permutations(fr_nonz.keys(), len(fr_nonz)):
    ans = do_valve_order(order, G, fr_nonz)
    print(ans)
    if ans > maxans:
        maxans = ans
        print(maxans)
#         print(order)
#     print('\n')
        
# print(maxans)

659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659


659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659


659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659


659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659


659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659


...

659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659


659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659


659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659


659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659
659


KeyboardInterrupt: 

In [101]:
maxans = 0

for target in fr_nonz:
    ans = 1
    t = 0
    for p in nx.all_paths(G, 'AA', target):
#     for p in nx.dfs_edges(G, 'AA'):
        print(p)
        for v in p[1:]:
            t += 1
            if t == 30:
                if ans > maxans:
                    maxans = ans
                break
            if v in fr_nonz:
                t += 1
                ans *= frs[v]
            if t==30:
                if ans > maxans:
                    maxans = ans

AttributeError: module 'networkx' has no attribute 'all_paths'

In [98]:
fr_nonz

{'HH': 22, 'JJ': 21, 'DD': 20, 'BB': 13, 'EE': 3, 'CC': 2}

In [96]:
maxans

0

In [87]:
# 8060774400 wrong

{'PM': 25,
 'XO': 23,
 'VI': 22,
 'II': 20,
 'KQ': 18,
 'TV': 17,
 'AS': 16,
 'HR': 14,
 'FQ': 12,
 'RU': 11,
 'MA': 10,
 'ED': 9,
 'MW': 7,
 'LF': 5,
 'DW': 4,
 'OX': 0,
 'GL': 0,
 'PQ': 0,
 'IR': 0,
 'QN': 0,
 'BP': 0,
 'ZY': 0,
 'CB': 0,
 'FR': 0,
 'IU': 0,
 'AA': 0,
 'CY': 0,
 'MT': 0,
 'GA': 0,
 'RR': 0,
 'AU': 0,
 'QP': 0,
 'CD': 0,
 'BD': 0,
 'HG': 0,
 'QD': 0,
 'LS': 0,
 'BT': 0,
 'QS': 0,
 'QM': 0,
 'MJ': 0,
 'AW': 0,
 'WJ': 0,
 'YZ': 0,
 'MO': 0,
 'BN': 0,
 'SA': 0,
 'EE': 0,
 'YC': 0,
 'KF': 0,
 'HY': 0,
 'YB': 0,
 'KP': 0,
 'FF': 0,
 'BL': 0,
 'RQ': 0,
 'FL': 0,
 'RH': 0,
 'FN': 0,
 'UG': 0,
 'WT': 0}